<a href="https://colab.research.google.com/github/shivhub09/IPD25/blob/main/IPD_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import spacy
from sentence_transformers import SentenceTransformer
from transformers import pipeline

model_name = "all-mpnet-base-v2"
sentence_encoder = SentenceTransformer(model_name)

nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
import numpy as np

In [2]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.6 MB/s eta 0:00:00


In [9]:
def detect_anomalies_sentence_embeddings(document):
  """
  This function detects anomalies using sentence embeddings and cosine similarity.

  Args:
      document: The text of the document.

  Returns:
      A list of sentence indices and their corresponding cosine similarity scores
      with the previous sentence (indicating potential anomalies).
  """
  sentences = document.split(". ")  # Split into sentences based on full stops
  sentence_vectors = sentence_encoder.encode(sentences)
  anomaly_scores = []

  for i in range(1, len(sentences)):
    similarity = np.dot(sentence_vectors[i - 1], sentence_vectors[i]) / (
        np.linalg.norm(sentence_vectors[i - 1]) * np.linalg.norm(sentence_vectors[i])
    )
    anomaly_scores.append((i, similarity))

  # Identify sentences with low similarity compared to neighbors (set your threshold)
  potential_anomalies = [
      score for score in anomaly_scores if score[1] < 0.6
  ]  # Adjust threshold as needed (e.g., 0.7 for stricter criteria)
  return potential_anomalies

def detect_anomalies_rule_based(document):
  """
  This function detects anomalies using rule-based checks for named entities.

  Args:
      document: The text of the document.

  Returns:
      A list of descriptions of identified rule-based anomalies.
  """
  doc = nlp(document)
  anomalies = []

  # Example rule 1: Check for contradictions in person names
  named_people = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
  for i in range(len(named_people) - 1):
    if named_people[i] != named_people[i + 1]:
      anomalies.append(f"Contradiction in person name: {named_people[i]} vs {named_people[i+1]}")

  # Add more rules here to check for other inconsistencies (e.g., numeric values)

  return anomalies


In [11]:
def main():
  # Your document text
  document = "John, a software engineer, is responsible for developing new features. However, in a later section, it is revealed he is actually a highly trained alien spy sent to infiltrate the company..."

  # Detect anomalies using sentence embeddings
  embedding_anomalies = detect_anomalies_sentence_embeddings(document)
  print("Sentence Embedding Anomalies:")
  for anomaly in embedding_anomalies:
    print(f"Sentence {anomaly[0] + 1} (Similarity: {anomaly[1]:.2f})")

  # Detect anomalies using rule-based approach
  rule_anomalies = detect_anomalies_rule_based(document)
  print("\nRule-Based Anomalies:")
  for anomaly in rule_anomalies:
    print(anomaly)

if __name__ == "__main__":
  main()


Sentence Embedding Anomalies:
Sentence 2 (Similarity: 0.26)

Rule-Based Anomalies:


# list of statements

In [23]:
import spacy
from nltk.corpus import stopwords

def preprocess_text(text):
  """
  This function preprocesses text data for anomaly detection.

  Args:
      text: The text string to be preprocessed.

  Returns:
      A list of preprocessed tokens (words).
  """
  # Lowercase the text
  text = text.lower()

  # Tokenize the text (split into words)
  words = text.split()

  # Remove punctuation (optional)
  punctuation = [".", ",", "!", "?", ";", ":"]
  words = [word for word in words if word not in punctuation]

  # Remove stop words (common words)
  stop_words = stopwords.words('english')
  words = [word for word in words if word not in stop_words]

  # Lemmatization (optional, reduces words to their base form)
  # lemmatizer = WordNetLemmatizer()
  # words = [lemmatizer.lemmatize(word) for word in words]

  return words


In [24]:
from sentence_transformers import SentenceTransformer

def encode_statement(statement):
  """
  This function preprocesses a statement and generates its sentence embedding.

  Args:
      statement: The text of the statement.

  Returns:
      A sentence embedding vector.
  """
  # Preprocess the statement
  preprocessed_text = preprocess_text(statement)

  # Encode the preprocessed statement
  model_name = "all-mpnet-base-v2"  # Replace with your chosen model
  model = SentenceTransformer(model_name)
  statement_vector = model.encode(preprocessed_text)
  return statement_vector


In [28]:
def detect_anomalies_similarity(statements):
  """
  This function detects anomalies in a list of statements using cosine similarity.

  Args:
      statements: A list of statement text strings.

  Returns:
      A list of indices of statements with potential contradictions.
  """
  # Encode each statement
  statement_vectors = [encode_statement(statement) for statement in statements]

  # Calculate cosine similarity matrix (fix the loop)
  similarity_matrix = np.zeros((len(statement_vectors), len(statement_vectors)))
  for i in range(len(statement_vectors)):
    for j in range(i, len(statement_vectors)):
      # Outer product for element-wise multiplication
      element_wise_product = np.outer(statement_vectors[i], statement_vectors[j])
      # Calculate norm products
      norm_product = np.linalg.norm(statement_vectors[i]) * np.linalg.norm(statement_vectors[j])
      # Avoid division by zero (set a small epsilon if necessary)
      similarity = np.sum(element_wise_product) / (norm_product + 1e-8)  # Add epsilon
      similarity_matrix[i, j] = similarity
      # No need to fill the lower triangular matrix as it's symmetric

  # Identify anomalous statements (low similarity with most others)
  threshold = 0.5  # Adjust threshold based on your data and desired strictness
  anomaly_indices = []
  for i in range(len(statement_vectors)):
    average_similarity = np.mean(similarity_matrix[i])
    if average_similarity < threshold:
      anomaly_indices.append(i)

  return anomaly_indices


In [29]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
import numpy as np

import numpy as np

def detect_anomalies_similarity(statements):
  """
  This function detects anomalies in a list of statements using cosine similarity.

  Args:
      statements: A list of statement text strings.

  Returns:
      A list of tuples, where each tuple contains the indices of two statements
      with potential contradictions.
  """
  # Encode each statement
  statement_vectors = [encode_statement(statement) for statement in statements]

  # Calculate cosine similarity matrix (element-wise)
  anomaly_pairs = []
  for i in range(len(statement_vectors)):
    for j in range(i + 1, len(statement_vectors)):  # Start j from i+1 (exclude self-similarity)
      # Outer product for element-wise multiplication
      element_wise_product = np.outer(statement_vectors[i], statement_vectors[j])
      # Calculate norm products
      norm_product = np.linalg.norm(statement_vectors[i]) * np.linalg.norm(statement_vectors[j])
      # Avoid division by zero (set a small epsilon if necessary)
      similarity = np.sum(element_wise_product) / (norm_product + 1e-8)

      # Identify potential anomalies based on threshold
      threshold = 0.5  # Adjust threshold based on your data and desired strictness
      if similarity < threshold:
        anomaly_pairs.append((i, j))  # Append indices of the anomaly pair

  return anomaly_pairs


# Example usage
statements = [
    "John, a software engineer, is responsible for developing new features.",
    "John is a data scientist with extensive experience in machine learning.",
    "The meeting will be held tomorrow at 10 AM in the conference room.",
    "The meeting will be held next week on Tuesday at 2 PM."
]

anomaly_pairs = detect_anomalies_similarity(statements)

if anomaly_pairs:
  print("Potential contradictions detected between:")
  for pair in anomaly_pairs:
    print(f"Statement {pair[0] + 1} and Statement {pair[1] + 1}")
else:
  print("No anomalies detected.")


Potential contradictions detected between:
Statement 1 and Statement 2
Statement 1 and Statement 3
Statement 1 and Statement 4
Statement 2 and Statement 3
Statement 2 and Statement 4
Statement 3 and Statement 4


In [36]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

# hybrid approach

In [42]:
import numpy as np

def get_keywords(statement):
  """
  This function extracts keywords from a statement using NLTK (replace with your preferred method).

  Args:
      statement: The text of the statement.

  Returns:
      A list of extracted keywords (lowercase).
  """

  words = nltk.word_tokenize(statement.lower())

  # Filter for nouns, verbs, and named entities (adjust based on your needs)
  keywords = [word for word in words if (
      nltk.pos_tag([word])[0][1] in ["NN", "NNP", "VB", "VBP"]
  )]
  return keywords

def detect_anomalies_hybrid(statements, your_keywords):
  """
  This function detects anomalies in a list of statements using cosine similarity
  and keyword heuristics.

  Args:
      statements: A list of statement text strings.
      your_keywords: A list of keywords relevant to the topic.

  Returns:
      A list of tuples, where each tuple contains the indices of two statements
      with potential contradictions.
  """
  # Encode each statement
  statement_vectors = [encode_statement(statement) for statement in statements]

  # Set minimum number of shared keywords for anomaly consideration
  min_keywords = 2  # Adjust threshold based on your data

  anomaly_pairs = []
  for i in range(len(statement_vectors)):
    for j in range(i + 1, len(statement_vectors)):
      # Check if statements share at least the minimum keywords
      shared_keywords = len(set(get_keywords(statements[i])) & set(get_keywords(statements[j])))
      if shared_keywords >= min_keywords:
        # Calculate cosine similarity
        element_wise_product = np.outer(statement_vectors[i], statement_vectors[j])
        norm_product = np.linalg.norm(statement_vectors[i]) * np.linalg.norm(statement_vectors[j])
        similarity = np.sum(element_wise_product) / (norm_product + 1e-8)

        # Identify potential anomalies based on threshold
        threshold = 0.5  # Adjust threshold based on your data and desired strictness
        if similarity < threshold:
          anomaly_pairs.append((i, j))

  return anomaly_pairs

# Example usage (replace with your actual statements)
statements = [
  "John has experience in reparing water tap because he is a plumber",
  "John is a data scientist with extensive experience in machine learning.",
  "The meeting will be held tomorrow at 10 AM in the conference room.",
  "The meeting  is scheduled for tommorrow at 10"
]

# Sample keywords related to meetings (adjust based on your specific topic)
your_keywords = ["meeting", "conference room", "reschedule", "john", "experience"]

anomaly_pairs = detect_anomalies_hybrid(statements, your_keywords)

if anomaly_pairs:
  print("Potential contradictions detected between:")
  for pair in anomaly_pairs:
    print(f"Statement {pair[0] + 1} and Statement {pair[1] + 1}")
else:
  print("No anomalies detected.")

Potential contradictions detected between:
Statement 1 and Statement 2
